In [69]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.neighbors import KNeighborsClassifier


## A. Data cleaning

In [2]:
data = pd.read_csv('data.csv')


/var/folders/2r/k8v37jcj6yg4bqx302w74hrm0000gn/T/ipykernel_91808/2471439888.py:1: DtypeWarning: Columns (5,8,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('data.csv')


In [3]:
data.isnull().sum()

fecha_dato                      0
ncodpers                        0
ind_empleado                27734
pais_residencia             27734
sexo                        27804
age                             0
fecha_alta                  27734
ind_nuevo                   27734
antiguedad                      0
indrel                      27734
ult_fec_cli_1t           13622516
indrel_1mes                149781
tiprel_1mes                149781
indresi                     27734
indext                      27734
conyuemp                 13645501
canal_entrada              186126
indfall                     27734
tipodom                     27735
cod_prov                    93591
nomprov                     93591
ind_actividad_cliente       27734
renta                     2794375
segmento                   189368
ind_ahor_fin_ult1               0
ind_aval_fin_ult1               0
ind_cco_fin_ult1                0
ind_cder_fin_ult1               0
ind_cno_fin_ult1                0
ind_ctju_fin_u

In [4]:
df_ind_empleado_null = data[data['ind_empleado'].isna()]
df_ind_empleado_null.to_csv("ind_empleado_null.csv", index=False)

* Ở cột ind_empleado và pais_residencia cứ dòng nào null là toàn bộ dữ liệu khách hàng đều null => drop hết null của 2 cột này

In [5]:
data = data.dropna(subset=['ind_empleado'])

* Đổi dữ liệu của cột indrel thành 1 và 0

In [6]:
data['indrel'].value_counts()

indrel
1.0     13594782
99.0       24793
Name: count, dtype: int64

In [7]:
data['indrel'] = data['indrel'].astype(int)
data['indrel'] = data['indrel'].replace(99, 0)

In [8]:
data['conyuemp'].value_counts()

conyuemp
N    1791
S      17
Name: count, dtype: int64

* Dữ liệu ở cột ult_fec_cli_1t null rất nhiều vì theo dữ liệu của cột indrel thì hầu hết khách hàng đều là khách hàng chính => bỏ cột này
* Dữ liệu ở cột conyuemp cũng null rất nhiều và lưu cả N(Không) và S(Có) chứ không lưu như mô tả dữ liệu => bỏ cột này vì không xử lí được

In [9]:
data = data.drop(columns=['ult_fec_cli_1t','conyuemp'])

* Các cột dữ liệu số (age, antiguedad) đang lưu dưới cả dạng object và float => chuyển hết về int
* Loại bỏ các giá trị vô lí (VD: age < 0, antiguedad <0, age > 117)

In [10]:
data['age'] = data['age'].astype(int)
data['antiguedad'] = data['antiguedad'].astype(int)

In [11]:
print(len(data[data['antiguedad']<0]))
print(len(data[data['age']<0]))

38
0


In [12]:
print(len(data[data['age']>117]))

22


In [13]:
data = data[data["antiguedad"] >= 0]

In [14]:
data = data[data["age"] <= 117]

In [15]:
data.isnull().sum()

fecha_dato                     0
ncodpers                       0
ind_empleado                   0
pais_residencia                0
sexo                          70
age                            0
fecha_alta                     0
ind_nuevo                      0
antiguedad                     0
indrel                         0
indrel_1mes               122047
tiprel_1mes               122047
indresi                        0
indext                         0
canal_entrada             158392
indfall                        0
tipodom                        1
cod_prov                   65857
nomprov                    65857
ind_actividad_cliente          0
renta                    2766626
segmento                  161634
ind_ahor_fin_ult1              0
ind_aval_fin_ult1              0
ind_cco_fin_ult1               0
ind_cder_fin_ult1              0
ind_cno_fin_ult1               0
ind_ctju_fin_ult1              0
ind_ctma_fin_ult1              0
ind_ctop_fin_ult1              0
ind_ctpp_f

In [16]:
data = data.dropna(subset=['sexo'])

* 2 cột indrel_1mes và tiprel_1mes null cùng nhau và chứa thông tin cũng khá giống nhau => có thể gộp 2 cột làm 1 để điền null 


In [17]:
data['indrel_1mes'] = data['indrel_1mes'].astype(str)
data['tiprel_1mes'] = data['tiprel_1mes'].astype(str)
data['indrel_1mes'] = data['indrel_1mes'].astype(str).str.replace(".0", "", regex=False)

In [18]:
print(data[['indrel_1mes','tiprel_1mes']].value_counts())

indrel_1mes  tiprel_1mes
1            I              7304607
             A              6185945
nan          nan             122047
3            P                 4350
2            A                 1082
P            R                  869
4            P                  306
2            I                  235
P            N                    4
Name: count, dtype: int64


In [ ]:
mapping = {
    ('1', 'I'): 'A',
    ('1', 'A'): 'B',
    ('3', 'P'): 'C',
    ('2', 'A'): 'D',
    ('P', 'R'): 'E',
    ('4', 'P'): 'F',
    ('2', 'I'): 'G',
    ('P', 'N'): 'E'  
}

data['inti_1mes'] = np.nan  

for idx, row in data.iterrows():
    key = (str(row['indrel_1mes']), str(row['tiprel_1mes']))
    
    if pd.isna(row['indrel_1mes']) and pd.isna(row['tiprel_1mes']):
        data.at[idx, 'inti_1mes'] = np.nan
    else:
        data.at[idx, 'inti_1mes'] = mapping.get(key, 'Unknown')

   
    if idx % 1000 == 0:  
        print(f"Processing index: {idx}")

data = data.drop(columns=['indrel_1mes', 'tiprel_1mes'])

data['inti_1mes'].value_counts()


In [20]:
mask = (data["inti_1mes"] == "Unknown")

data.loc[mask & (data["ind_actividad_cliente"] == 1), "inti_1mes"] = "B"
data.loc[mask & (data["ind_actividad_cliente"] == 0), "inti_1mes"] = "A"

In [21]:
data['inti_1mes'].value_counts()

inti_1mes
A    7388380
B    6224219
C       4350
D       1082
E        873
F        306
G        235
Name: count, dtype: int64

In [23]:
columns_to_drop = ['pais_residencia','fecha_alta','ind_nuevo','canal_entrada','tipodom','nomprov','ind_actividad_cliente']
data = data.drop(columns = columns_to_drop)

In [25]:
data["ind_nomina_ult1"].fillna(0, inplace=True)
data["ind_nom_pens_ult1"].fillna(0, inplace=True)


/var/folders/2r/k8v37jcj6yg4bqx302w74hrm0000gn/T/ipykernel_91808/1560314996.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data["ind_nomina_ult1"].fillna(0, inplace=True)
/var/folders/2r/k8v37jcj6yg4bqx302w74hrm0000gn/T/ipykernel_91808/1560314996.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values alway

In [27]:
data["segmento"].fillna(data["segmento"].mode()[0], inplace=True)
data["cod_prov"].fillna(0, inplace=True)


/var/folders/2r/k8v37jcj6yg4bqx302w74hrm0000gn/T/ipykernel_91808/2535807846.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data["segmento"].fillna(data["segmento"].mode()[0], inplace=True)
/var/folders/2r/k8v37jcj6yg4bqx302w74hrm0000gn/T/ipykernel_91808/2535807846.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are se

In [28]:
data.isnull().sum()

fecha_dato                 0
ncodpers                   0
ind_empleado               0
sexo                       0
age                        0
antiguedad                 0
indrel                     0
indresi                    0
indext                     0
indfall                    0
cod_prov                   0
renta                2766592
segmento                   0
ind_ahor_fin_ult1          0
ind_aval_fin_ult1          0
ind_cco_fin_ult1           0
ind_cder_fin_ult1          0
ind_cno_fin_ult1           0
ind_ctju_fin_ult1          0
ind_ctma_fin_ult1          0
ind_ctop_fin_ult1          0
ind_ctpp_fin_ult1          0
ind_deco_fin_ult1          0
ind_deme_fin_ult1          0
ind_dela_fin_ult1          0
ind_ecue_fin_ult1          0
ind_fond_fin_ult1          0
ind_hip_fin_ult1           0
ind_plan_fin_ult1          0
ind_pres_fin_ult1          0
ind_reca_fin_ult1          0
ind_tjcr_fin_ult1          0
ind_valo_fin_ult1          0
ind_viv_fin_ult1           0
ind_nomina_ult

In [41]:
data['cod_prov'] = data['cod_prov'].astype(int)

In [44]:
data['ind_nomina_ult1'] = data['ind_nomina_ult1'].astype(int)
data['ind_nom_pens_ult1'] = data['ind_nom_pens_ult1'].astype(int)

In [47]:
data.to_csv('data_for_visualize.csv',index = False)

In [81]:
data = pd.read_csv('data_for_visualize.csv')

In [82]:
label_cols = ['sexo', 'indresi', 'indext', 'indfall']
le = LabelEncoder()
for col in label_cols:
    data[col] = le.fit_transform(data[col])

In [83]:
data = pd.get_dummies(data, columns=['ind_empleado'], drop_first=True)

In [84]:
cols_to_convert = ['ind_empleado_B', 'ind_empleado_F', 'ind_empleado_N','ind_empleado_S']  
data[cols_to_convert] = data[cols_to_convert].astype(int)

In [85]:
mapping = {"B": 6, "A": 5, "D": 4, "G": 3, "E": 2, "C": 1, "F": 0}
data["inti_1mes"] = data["inti_1mes"].map(mapping)


In [86]:
mapping = {"01 - TOP": 2, "02 - PARTICULARES": 1, "03 - UNIVERSITARIO": 0}
data["segmento"] = data["segmento"].map(mapping)


In [90]:
data_scale = data.copy()
data_scale = data_scale.drop(columns=['ncodpers', 'fecha_dato'])
features = [col for col in data_scale.columns]
# Scale data
scaler = MinMaxScaler()
data_scale[features] = scaler.fit_transform(data_scale[features])


In [93]:
data_scale.shape

(13619445, 39)

In [94]:
fill_nan_columns = ['renta']
col_means = data_scale[fill_nan_columns].mean()
imputer = KNNImputer(n_neighbors=3)
batch_size = 100000  
nums_batch = len(data_scale) // batch_size + 1
imputed_batches = []
for i in range(0, len(data_scale), batch_size):
    print(f"{i}/{nums_batch}")
    batch = data_scale.iloc[i:i + batch_size]
    
    imputed_batch = imputer.fit_transform(batch[fill_nan_columns])
    imputed_batches.append(pd.DataFrame(imputed_batch, columns=fill_nan_columns))
    
# Ghép các batch lại
train_imputed = pd.concat(imputed_batches, ignore_index=True)
for col in data_scale.columns:
    if col not in fill_nan_columns:
        train_imputed.loc[:, col] = data_scale.loc[:, col]
        
data_scale = train_imputed

0/137
100000/137
200000/137
300000/137
400000/137
500000/137
600000/137
700000/137
800000/137
900000/137
1000000/137
1100000/137
1200000/137
1300000/137
1400000/137
1500000/137
1600000/137
1700000/137
1800000/137
1900000/137
2000000/137
2100000/137
2200000/137
2300000/137
2400000/137
2500000/137
2600000/137
2700000/137
2800000/137
2900000/137
3000000/137
3100000/137
3200000/137
3300000/137
3400000/137
3500000/137
3600000/137
3700000/137
3800000/137
3900000/137
4000000/137
4100000/137
4200000/137
4300000/137
4400000/137
4500000/137
4600000/137
4700000/137
4800000/137
4900000/137
5000000/137
5100000/137
5200000/137
5300000/137
5400000/137
5500000/137
5600000/137
5700000/137
5800000/137
5900000/137
6000000/137
6100000/137
6200000/137
6300000/137
6400000/137
6500000/137
6600000/137
6700000/137
6800000/137
6900000/137
7000000/137
7100000/137
7200000/137
7300000/137
7400000/137
7500000/137
7600000/137
7700000/137
7800000/137
7900000/137
8000000/137
8100000/137
8200000/137
8300000/137
8400000

In [97]:
data_scale_1 = data_scale.copy()

In [99]:
data_scale.columns

Index(['renta', 'sexo', 'age', 'antiguedad', 'indrel', 'indresi', 'indext',
       'indfall', 'cod_prov', 'segmento', 'ind_ahor_fin_ult1',
       'ind_aval_fin_ult1', 'ind_cco_fin_ult1', 'ind_cder_fin_ult1',
       'ind_cno_fin_ult1', 'ind_ctju_fin_ult1', 'ind_ctma_fin_ult1',
       'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1', 'ind_deco_fin_ult1',
       'ind_deme_fin_ult1', 'ind_dela_fin_ult1', 'ind_ecue_fin_ult1',
       'ind_fond_fin_ult1', 'ind_hip_fin_ult1', 'ind_plan_fin_ult1',
       'ind_pres_fin_ult1', 'ind_reca_fin_ult1', 'ind_tjcr_fin_ult1',
       'ind_valo_fin_ult1', 'ind_viv_fin_ult1', 'ind_nomina_ult1',
       'ind_nom_pens_ult1', 'ind_recibo_ult1', 'inti_1mes', 'ind_empleado_B',
       'ind_empleado_F', 'ind_empleado_N', 'ind_empleado_S'],
      dtype='object')

In [100]:
data_scale[features] = scaler.inverse_transform(data_scale[features])

In [155]:
int_col = data_scale.columns[1:39]

In [150]:
data_scale[int_col] = data_scale[int_col].astype(int)

In [152]:
data_scale.dtypes

renta                float64
sexo                   int64
age                    int64
antiguedad             int64
indrel                 int64
indresi                int64
indext                 int64
indfall                int64
cod_prov               int64
segmento               int64
ind_ahor_fin_ult1      int64
ind_aval_fin_ult1      int64
ind_cco_fin_ult1       int64
ind_cder_fin_ult1      int64
ind_cno_fin_ult1       int64
ind_ctju_fin_ult1      int64
ind_ctma_fin_ult1      int64
ind_ctop_fin_ult1      int64
ind_ctpp_fin_ult1      int64
ind_deco_fin_ult1      int64
ind_deme_fin_ult1      int64
ind_dela_fin_ult1      int64
ind_ecue_fin_ult1      int64
ind_fond_fin_ult1      int64
ind_hip_fin_ult1       int64
ind_plan_fin_ult1      int64
ind_pres_fin_ult1      int64
ind_reca_fin_ult1      int64
ind_tjcr_fin_ult1      int64
ind_valo_fin_ult1      int64
ind_viv_fin_ult1       int64
ind_nomina_ult1        int64
ind_nom_pens_ult1      int64
ind_recibo_ult1        int64
inti_1mes     

In [154]:
data_scale.to_csv('clean.csv',index = False)